In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [8]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')
df_jan2023 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb2023 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [9]:
len(df_jan2023.columns) #number of columns (question 1)

19

In [11]:
df_jan2023['duration'] = df_jan2023['tpep_dropoff_datetime'] - df_jan2023['tpep_pickup_datetime']
df_jan2023.duration = df_jan2023.duration.apply(lambda td: td.total_seconds() / 60)
df_jan2023.duration.std() #standard deviation of the trips duration in January (Question 2)

42.594351241920904

In [12]:
df_jan2023_filt = df_jan2023[(df_jan2023.duration >= 1) & (df_jan2023.duration <= 60)]
perc = 100 - ((len(df_jan2023.axes[0]) - len(df_jan2023_filt.axes[0]))/len(df_jan2023.axes[0]) *100)
perc #fraction of the records left after you dropped the outliers (Question 3)

98.1220282212598

In [13]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_jan2023_filt[categorical] = df_jan2023_filt[categorical].astype(str)

/tmp/ipykernel_18245/1964931108.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan2023_filt[categorical] = df_jan2023_filt[categorical].astype(str)


In [15]:
def is_string_series(s : pd.Series):
    if isinstance(s.dtype, pd.StringDtype):
        # The series was explicitly created as a string series (Pandas>=1.0.0)
        return True
    elif s.dtype == 'object':
        # Object series, check each value
        return all((v is None) or isinstance(v, str) for v in s)
    else:
        return False

print(is_string_series(df_jan2023_filt['PULocationID']))
print(is_string_series(df_jan2023_filt['DOLocationID']))

True
True


In [18]:
train_dicts = df_jan2023_filt[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df_jan2023_filt[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False) #RMSE on train (Question 5)

7.649143388169879

In [21]:
X_train #dimensionality of this matrix, number of columns (Question 4)

<3009173x516 sparse matrix of type '<class 'numpy.float64'>'
	with 9027519 stored elements in Compressed Sparse Row format>